In [1]:
import arcpy
import psycopg2

In [13]:
pointArray = arcpy.Array([arcpy.Point(0,0),arcpy.Point(0,10),arcpy.Point(10,10),arcpy.Point(10,0)])

polygon = arcpy.Polygon(pointArray)

wgs84 = arcpy.SpatialReference(4326)

polygon.SR = wgs84

polygon_wkt = polygon.WKT

In [14]:
connection = psycopg2.connect(host = '35.193.80.209',
                             database = 'lab0',
                             user = 'postgres',
                             password = '')

cursor = connection.cursor()

#create table
create_table_query = sql.SQL("CREATE TABLE myPolygon (id integer, name Text, geog GEOMETRY(MultiPOLYGON,4326))")

cursor.execute(create_table_query)

#insert polygon
sql = f'''
INSERT INTO myPolygon VALUES (1, 'example_poly',ST_GeomFromWKB(ST_AsEWKB(ST_GeomFromText('{polygon_wkt}')),4326));
'''

cursor.execute(sql)

connection.commit()

cursor.close()

connection.close()

In [14]:
import requests

url = 'http://34.125.39.137:5000/get_polygon'
response = requests.get(url)

if response.status_code == 200:
    geojson = response.json()
    with open("C:/Users/yaxua/Desktop/myGeojson.geojson",'w') as f:
        f.write(str(geojson))
        print('GeoJSON file saved')
else:
    print('Failed')

GeoJSON file saved


In [39]:
import json

# Load the JSON string as a Python object
json_obj_1 = geojson

# Convert the first JSON object to the second JSON object
json_obj_2 = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "MultiPolygon",
            "geometry": json_obj_1['geometry'],
            "properties": {
                "name": "ExamplePolygon"
            }
        }
    ]
}

# Convert the Python object to a JSON string
json_string_2 = json.dumps(json_obj_2)

# Print the resulting JSON string
print(json_string_2)

# Parse the JSON string into a dictionary
json_dict = json.loads(json_string_2)

# Save the dictionary as a JSON file
with open("C:/Users/yaxua/Desktop/myEsriGeojson.geojson", 'w') as json_file:
    json.dump(json_dict, json_file)

{"type": "FeatureCollection", "features": [{"type": "MultiPolygon", "geometry": {"coordinates": [[[[0, 10.00012207], [10.00012207, 0], [10.00012207, 10.00012207], [0, 10.00012207]]]], "type": "MultiPolygon"}, "properties": {"name": "ExamplePolygon"}}]}


In [46]:
import arcgis
gis = arcgis.gis.GIS("home")

In [49]:
# File previously loaded into AGOL (manually)
file_path = r"C:/Users/yaxua/Desktop/myEsriGeojson.geojson"

# Add item 
props = {"title":"myGeojson", "description":"Lab 1.2", "tags":"GIS 5572", "type":"GeoJson"}
item = gis.content.add(item_properties=props, data=file_path)



In [ ]:
# Publish item
item.publish(None)

feature_layer_collection = FeatureLayerCollection.fromitem(item)